In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
from mne.datasets import eegbci
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense, LSTM, Reshape, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import LeaveOneGroupOut
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

# =============================================================================
# 1. MEMUAT DATA & MEMBUAT GRUP SUBJEK
# =============================================================================
print("Memuat data dari semua subjek...")

subjects = range(1, 21)
all_X = []
all_y = []
groups = []

for subject_id in subjects:
    print(f"  Memproses subjek {subject_id}...")
    runs = [6, 10, 14]
    raw_files = eegbci.load_data(subject_id, runs, update_path=True, verbose=False)
    raw = mne.io.concatenate_raws([mne.io.read_raw_edf(f, preload=True, verbose='WARNING') for f in raw_files])

    raw.pick_types(meg=False, eeg=True, stim=False, eog=False, exclude='bads')
    raw.filter(l_freq=4.0, h_freq=38.0, fir_design='firwin', skip_by_annotation='edge')

    event_id = dict(T1=1, T2=2)
    events, _ = mne.events_from_annotations(raw, event_id=event_id)

    tmin, tmax = -1., 4.
    epochs = mne.Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax,
                        proj=True, baseline=None, preload=True, verbose=False)

    X_subject = epochs.get_data()
    y_subject = epochs.events[:, -1] - 1

    all_X.append(X_subject)
    all_y.append(y_subject)
    groups.append(np.full(len(X_subject), subject_id))

X = np.concatenate(all_X, axis=0)
y = np.concatenate(all_y, axis=0)
groups = np.concatenate(groups, axis=0)

print(f"\n✅ Data berhasil digabungkan. Bentuk X: {X.shape}, Bentuk y: {y.shape}")

# =============================================================================
# 2. FUNGSI UNTUK MEMBUAT MODEL
# =============================================================================
def build_model(n_channels, n_timesteps, n_classes):
    inputs = Input(shape=(n_channels, n_timesteps, 1))
    x = Conv2D(16, (1, 16), activation='elu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(32, (n_channels, 1), activation='elu', padding='valid')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    target_shape = (n_timesteps, 32)
    x = Reshape(target_shape)(x)
    x = LSTM(128, return_sequences=False)(x)
    x = Dropout(0.5)(x)
    outputs = Dense(n_classes, activation='softmax')(x)
    model = Model(inputs, outputs)

    custom_optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# =============================================================================
# 3. TAHAP EVALUASI: LOOP VALIDASI SILANG LEAVE-ONE-SUBJECT-OUT (LOSO-CV)
# =============================================================================
logo = LeaveOneGroupOut()
scores = []

for fold, (train_idx, test_idx) in enumerate(logo.split(X, y, groups)):
    print(f"\n===== FOLD EVALUASI {fold+1}/{len(subjects)} =====")

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    test_subject = groups[test_idx][0]
    print(f"Training on {len(np.unique(groups[train_idx]))} subjects, Testing on subject {test_subject}")

    scaler = StandardScaler()
    X_train_reshaped = X_train.reshape(len(X_train), -1)
    X_test_reshaped = X_test.reshape(len(X_test), -1)
    scaler.fit(X_train_reshaped)
    X_train_norm = scaler.transform(X_train_reshaped).reshape(X_train.shape)
    X_test_norm = scaler.transform(X_test_reshaped).reshape(X_test.shape)

    X_train_final = np.expand_dims(X_train_norm, axis=-1)
    X_test_final = np.expand_dims(X_test_norm, axis=-1)
    y_train_final = to_categorical(y_train)
    y_test_final = to_categorical(y_test)

    n_channels = X_train_final.shape[1]
    n_timesteps = X_train_final.shape[2]
    n_classes = y_train_final.shape[1]

    model = build_model(n_channels, n_timesteps, n_classes)

    # KOREKSI: Cukup gunakan EarlyStopping. ModelCheckpoint tidak diperlukan di sini.
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

    model.fit(
        X_train_final, y_train_final,
        batch_size=16,
        epochs=100,
        callbacks=[early_stopping],
        validation_data=(X_test_final, y_test_final),
        verbose=2
    )

    loss, accuracy = model.evaluate(X_test_final, y_test_final, verbose=0)
    scores.append(accuracy)
    print(f"Accuracy on subject {test_subject}: {accuracy * 100:.2f}%")

# =============================================================================
# 4. HASIL EVALUASI AKHIR
# =============================================================================
mean_accuracy = np.mean(scores)
std_accuracy = np.std(scores)

print("\n===== HASIL AKHIR VALIDASI SILANG =====")
print(f"Akurasi Rata-rata: {mean_accuracy * 100:.2f}%")
print(f"Standar Deviasi Akurasi: {std_accuracy * 100:.2f}%")

plt.figure(figsize=(10, 6))
plt.boxplot(scores)
plt.title('Distribusi Akurasi per Subjek (LOSO-CV)')
plt.ylabel('Akurasi')
plt.xlabel('Model CNN-LSTM')
plt.ylim(0, 1)
plt.grid(True)
plt.show()

# =============================================================================
# 5. TAHAP PELATIHAN FINAL
# KOREKSI: Latih satu model terakhir menggunakan SEMUA data.
# =============================================================================
print("\n===== MEMULAI PELATIHAN MODEL FINAL MENGGUNAKAN SEMUA DATA =====")

# 1. Normalisasi menggunakan semua data
scaler_final = StandardScaler()
X_reshaped_final = X.reshape(len(X), -1)
X_norm_final = scaler_final.fit_transform(X_reshaped_final).reshape(X.shape)

# 2. Siapkan data untuk model
X_final_for_training = np.expand_dims(X_norm_final, axis=-1)
y_final_for_training = to_categorical(y)

# 3. Bangun dan Latih model final
n_channels = X_final_for_training.shape[1]
n_timesteps = X_final_for_training.shape[2]
n_classes = y_final_for_training.shape[1]

final_model = build_model(n_channels, n_timesteps, n_classes)

# Latih untuk beberapa epoch saja, karena tidak ada set validasi untuk early stopping
# Tujuannya adalah mengekspos model ke semua data yang ada.
final_model.fit(
    X_final_for_training, y_final_for_training,
    batch_size=16,
    epochs=15, # Latih untuk jumlah epoch yang wajar
    verbose=1
)

# 4. Simpan model final untuk portofolio Anda
final_model.save('bci_cnn_lstm_final.keras')
print("\n✅ Model final berhasil dilatih dan disimpan sebagai 'bci_cnn_lstm_final.keras'")